<a href="https://colab.research.google.com/github/Bahadir-OZCANLI/DecisionSupportSystem/blob/main/english_speak%26interview_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Setup and controls.

In [1]:
!pip install -q kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [2]:
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration
import numpy as np
import openai
import gradio as gr

import librosa
from kokoro import KPipeline
#from IPython.display import display, Audio
import soundfile as sf
import torch
import scipy.io.wavfile as wavfile

In [3]:
hf_token=userdata.get('HF_TOKEN')
openai.api_key = userdata.get("OPENAI_API_KEY")

if hf_token and hf_token.startswith("hf_"):
  print("HF key looks good so far")
else:
  print("HF key is not set - please click the key in the left sidebar")
login(hf_token, add_to_git_credential=True)

if openai.api_key:
  print("OpenAI key looks good so far")
else:
    print("openai_api_key is not set")
MODEL = "gpt-4.1-mini"

HF key looks good so far
OpenAI key looks good so far


In [4]:
# speech-to-text model
# Load the model and processor explicitly
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
# Move to GPU if available
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# text-to-speech pipeline
tts_pipeline = KPipeline(lang_code='a')

preprocessor_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/167 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


kokoro-v1_0.pth:   0%|          | 0.00/327M [00:00<?, ?B/s]

###Speech Recognation Model function.

In [5]:
def speech_to_text(filepath):
  """ Speech to text model """
  if filepath is None:
        return "No audio file."

  # 1. Load audio (Standard 16k for Whisper)
  audio, rate = librosa.load(filepath, sr=16000)

  # 2. Process the audio into "input features"
  # This replaces the pipeline's internal confusing logic
  inputs = processor(audio, sampling_rate=rate, return_tensors="pt")

  # 3. Move inputs to the GPU (if you are using one)
  input_features = inputs.input_features.to(device)

  # 4. Generate the transcription IDs
  generated_ids = model.generate(input_features)

  # 5. Decode the IDs back into text
  transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return transcription

### Text to Speech Model function.

In [6]:
def text_to_speech(text):
  """ Text to speech model. """
  generator = tts_pipeline(text, voice='af_heart')

  # 1. Initialize a list to hold all audio pieces
  all_audio_pieces = []

  # 2. Iterate through the generator
  for i, (gs, ps, audio) in enumerate(generator):
    # 'audio' is a numpy array
    all_audio_pieces.append(audio)

  # Optional: Keep this if you want to see debug info in the logs
  # print(f"Generated chunk {i}")

  # 3. Combine all pieces into one long audio array
  if not all_audio_pieces:
      return None
  full_audio = np.concatenate(all_audio_pieces)

  # 4. Save to a single file (Gradio needs a file path)
  output_path = 'final_response.wav'
  sf.write(output_path, full_audio, 24000)

  # Return the path string
  return output_path

###ChatGpt chat function.

In [12]:
sm_interview = """
  ROLE: You are an experienced Technical Interviewer for a junior LLM Engineering position.
  CONTENT: The candidate is a beginner taking Ed Donner's LLM Engineering course.
  GOAL: Evaluate the candidate's technical knowledge of basic LLM Engineering.

  CONSTRAINTS:
    - **Topic Scope:** Only ask questions about basic LLM concepts, Transformers, Hugging Face, and Pipelines. DO NOT ask about RAG or Fine-Tuning.
    - **Progressive Difficulty:** Start with a very easy question and make each subsequent question slightly harder.
    - **Keep it Short:** Your responses MUST be UNDER 60 WORDS. This is a voice conversation.
    - **One by One:** Never ask two questions in the same turn.
    - **Simplicity:** Use simple vocabulary suited for an intermediate English speaker.

  INSTRUCTIONS:
    1. **Interaction Loop (Max 10 Questions):** - Ask one technical question.
        - Wait for the user's response.
        - Briefly and supportively correct any wrong answers.
        - Ask the next question.
    2. **Completion:** After exactly 3 questions have been answered, DO NOT ask another question. Thank the candidate, give them a brief 1-2 sentence evaluation of their performance, and end the interview.
"""

In [8]:
system_message = """
ROLE: You are an experienced, friendly English teacher for non-native speakers (specifically Turkish speakers).
GOAL: Engage the user in a spoken conversation. Help them practice speaking by asking questions about daily life.

INSTRUCTIONS:
1.  **Topic Selection:** Choose a simple daily topic (e.g., Food, Hobbies, Family, Travel).
2.  **Interaction Loop:** - Ask ONE simple question at a time.
    - Wait for the user's response.
    - Briefly correct any major grammar mistakes in a supportive way.
    - Then, ask a follow-up question or move to the next topic.
3.  **Completion:** After about 10 exchanges, thank the user and give a short evaluation of their speaking level.

CONSTRAINTS:
- **Keep it Short:** Your responses must be UNDER 40 WORDS. This is a voice conversation; long texts are boring to listen to.
- **One by One:** Never ask two questions in the same turn.
- **Simplicity:** Use simple vocabulary suited for a beginner/intermediate level.
"""

In [13]:
history = []
def chat(message):
  """ Chat to ChatGpt """
  history.append({"role":"user", "content":message})
  messages = [{"role": "system", "content": sm_interview}] + history
  response = openai.chat.completions.create(model=MODEL, messages=messages)
  assistant = response.choices[0].message.content
  history.append({"role":"assistant", "content":assistant})

  return assistant

###Merge all the functions and run the gradio.

In [10]:
def conversation_pipeline(audio_input):
  """
  This is the main function to run all.
  """
  if audio_input is None:
      return "Please record your voice first."

  # 1. Transcribe
  user_text = speech_to_text(audio_input)

  # 2. Get AI text response
  ai_response_text = chat(user_text)

  # 3. Generate Audio and get the FILE PATH
  ai_audio_path = text_to_speech(ai_response_text)

  # 4. Return the path to Gradio
  return ai_audio_path

In [14]:
demo = gr.Interface(
    fn=conversation_pipeline,
    inputs = gr.Audio(type="filepath"),
    outputs = gr.Audio(type="numpy")
)
demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fe2a8f2a59b36aadc7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fe2a8f2a59b36aadc7.gradio.live
